# 1.8c: Black Hole L∞ Distances

We discovered 4 black holes sitting at discrete lattice points:
1. (x=-ε, y=0, z=0): 866 tokens
2. (x=+ε, y=+ε, z=0): 734 tokens
3. (x=+ε, y=0, z=0): 329 tokens
4. (x=-ε, y=+ε, z=0): 249 tokens

Where ε ≈ 1.526e-5.

**Question:** What are the L∞ (max component-wise) distances between these 4 points?

This will tell us if they're separated by exactly 1 ULP in bfloat16.

## Parameters

In [1]:
# Model to analyze
MODEL_NAME = "Qwen3-4B-Instruct-2507"

## Imports

In [2]:
import torch
import ml_dtypes
import numpy as np
from safetensors.torch import load_file
from pathlib import Path
from collections import defaultdict

# Helper function
def torch_bf16_to_numpy_bf16(tensor):
    """Convert PyTorch bfloat16 tensor to numpy array with ml_dtypes.bfloat16 dtype."""
    return tensor.cpu().view(torch.uint16).numpy().view(ml_dtypes.bfloat16)

## Load Core and Reconstruct Black Holes

In [3]:
# Load W in bfloat16
W_path = Path(f"../tensors/{MODEL_NAME}/W.safetensors")
W_bf16 = load_file(W_path)["W"]

# Load core data
core_path = Path(f"../tensors/{MODEL_NAME}/1.8a_core.safetensors")
core_data = load_file(core_path)

core_mask = core_data["core_mask"].to(torch.bool)
n_core = core_data["n_core"].item()

# Basis indices
north_idx = core_data["north_idx"].item()
meridian_idx = core_data["meridian_idx"].item()
equinox_idx = core_data["equinox_idx"].item()

print(f"Loaded core: {n_core:,} tokens")
print(f"Basis dimensions: {meridian_idx}, {north_idx}, {equinox_idx}")

Loaded core: 2,179 tokens
Basis dimensions: 322, 163, 1564


## Compute Centered Coordinates (bfloat16)

In [4]:
print("\nComputing centered coordinates...\n")

# Extract core and center
W_core_bf16 = W_bf16[core_mask]
core_centroid_bf16 = W_core_bf16.mean(dim=0)
W_core_centered_bf16 = W_core_bf16 - core_centroid_bf16

# Extract coordinates
x_bf16 = W_core_centered_bf16[:, meridian_idx]
y_bf16 = W_core_centered_bf16[:, equinox_idx]
z_bf16 = W_core_centered_bf16[:, north_idx]

# Compute r²
r_squared_bf16 = x_bf16**2 + y_bf16**2 + z_bf16**2

print("✓ Computed centered coordinates in bfloat16")


Computing centered coordinates...

✓ Computed centered coordinates in bfloat16


## Group by Coordinates to Find Black Holes

In [5]:
print("\nGrouping tokens by coordinates...\n")

# Create coordinate tuples
coords_bf16 = torch.stack([r_squared_bf16, x_bf16, y_bf16, z_bf16], dim=1)
coords_np_bf16 = torch_bf16_to_numpy_bf16(coords_bf16)

# Group tokens
coord_groups = defaultdict(list)
for i in range(n_core):
    coord_tuple = tuple(coords_np_bf16[i])
    coord_groups[coord_tuple].append(i)

# Get black holes (groups with >1 token)
black_holes = [(coord, indices) for coord, indices in coord_groups.items() if len(indices) > 1]
black_holes.sort(key=lambda x: len(x[1]), reverse=True)

print(f"Found {len(black_holes)} black holes")
print()
for i, (coord, indices) in enumerate(black_holes, 1):
    r2, x, y, z = coord
    print(f"Black hole {i}: {len(indices):4,} tokens at (x={float(x):.6e}, y={float(y):.6e}, z={float(z):.6e})")


Grouping tokens by coordinates...

Found 4 black holes

Black hole 1:  866 tokens at (x=-1.525879e-05, y=0.000000e+00, z=0.000000e+00)
Black hole 2:  734 tokens at (x=1.525879e-05, y=1.525879e-05, z=0.000000e+00)
Black hole 3:  329 tokens at (x=1.525879e-05, y=0.000000e+00, z=0.000000e+00)
Black hole 4:  249 tokens at (x=-1.525879e-05, y=1.525879e-05, z=0.000000e+00)


## Extract Black Hole Centers as Vectors

In [6]:
print("\nExtracting black hole center coordinates...\n")

# Extract (x, y, z) for each black hole (in bfloat16)
bh_centers_bf16 = np.array([[coord[1], coord[2], coord[3]] for coord, _ in black_holes])

n_bh = len(black_holes)
print(f"Black hole centers (bfloat16):")
print(f"{'BH':<4} {'x':<15} {'y':<15} {'z'}")
print("-" * 50)
for i in range(n_bh):
    x, y, z = bh_centers_bf16[i]
    print(f"{i+1:<4} {float(x):<15.6e} {float(y):<15.6e} {float(z):.6e}")


Extracting black hole center coordinates...

Black hole centers (bfloat16):
BH   x               y               z
--------------------------------------------------
1    -1.525879e-05   0.000000e+00    0.000000e+00
2    1.525879e-05    1.525879e-05    0.000000e+00
3    1.525879e-05    0.000000e+00    0.000000e+00
4    -1.525879e-05   1.525879e-05    0.000000e+00


## Compute Pairwise L∞ Distances

In [7]:
print("\nComputing pairwise L∞ distances...\n")

# Compute L∞ distances (max component-wise difference)
# For each pair (i, j), compute max(|x_i - x_j|, |y_i - y_j|, |z_i - z_j|)

print(f"Pairwise L∞ distances:")
print(f"{'BH i':<6} {'BH j':<6} {'L∞ distance':<18} {'Dimensions where max occurs'}")
print("-" * 70)

for i in range(n_bh):
    for j in range(i+1, n_bh):
        # Compute component-wise absolute differences
        diff = np.abs(bh_centers_bf16[i] - bh_centers_bf16[j])
        
        # L∞ is the maximum component
        linf_dist = float(np.max(diff))
        
        # Find which dimensions have the max
        max_dims = []
        for dim_idx, (dim_name, dim_val) in enumerate(zip(['x', 'y', 'z'], diff)):
            if float(dim_val) == linf_dist:
                max_dims.append(dim_name)
        
        dims_str = ', '.join(max_dims)
        print(f"{i+1:<6} {j+1:<6} {linf_dist:<18.10e} {dims_str}")


Computing pairwise L∞ distances...

Pairwise L∞ distances:
BH i   BH j   L∞ distance        Dimensions where max occurs
----------------------------------------------------------------------
1      2      3.0517578125e-05   x
1      3      3.0517578125e-05   x
1      4      1.5258789062e-05   y
2      3      1.5258789062e-05   y
2      4      3.0517578125e-05   x
3      4      3.0517578125e-05   x


## Full Distance Matrix

In [8]:
print("\nL∞ distance matrix:")
print()

# Build full matrix
dist_matrix = np.zeros((n_bh, n_bh))
for i in range(n_bh):
    for j in range(n_bh):
        if i == j:
            dist_matrix[i, j] = 0.0
        else:
            diff = np.abs(bh_centers_bf16[i] - bh_centers_bf16[j])
            dist_matrix[i, j] = float(np.max(diff))

# Print matrix
print("     BH1         BH2         BH3         BH4")
for i in range(n_bh):
    row_str = f"BH{i+1} "
    for j in range(n_bh):
        row_str += f"{dist_matrix[i, j]:>11.4e} "
    print(row_str)


L∞ distance matrix:

     BH1         BH2         BH3         BH4
BH1  0.0000e+00  3.0518e-05  3.0518e-05  1.5259e-05 
BH2  3.0518e-05  0.0000e+00  1.5259e-05  3.0518e-05 
BH3  3.0518e-05  1.5259e-05  0.0000e+00  3.0518e-05 
BH4  1.5259e-05  3.0518e-05  3.0518e-05  0.0000e+00 


## Analysis: ULP Separation

In [9]:
print("\n" + "="*60)
print("ANALYSIS: ULP SEPARATION")
print("="*60)
print()

# Get unique L∞ distances
unique_dists = np.unique(dist_matrix[dist_matrix > 0])

print(f"Unique L∞ distances between black holes:")
for i, dist in enumerate(unique_dists, 1):
    print(f"  {i}. {dist:.10e}")
print()

# Check if they're integer multiples
if len(unique_dists) > 1:
    min_dist = unique_dists[0]
    print(f"Minimum distance: {min_dist:.10e}")
    print(f"\nRatios to minimum distance:")
    for dist in unique_dists:
        ratio = dist / min_dist
        print(f"  {dist:.10e} / {min_dist:.10e} = {ratio:.2f}")
    print()
    print("If ratios are integers, black holes are on a regular lattice!")

print()
print("="*60)


ANALYSIS: ULP SEPARATION

Unique L∞ distances between black holes:
  1. 1.5258789062e-05
  2. 3.0517578125e-05

Minimum distance: 1.5258789062e-05

Ratios to minimum distance:
  1.5258789062e-05 / 1.5258789062e-05 = 1.00
  3.0517578125e-05 / 1.5258789062e-05 = 2.00

If ratios are integers, black holes are on a regular lattice!

